# Information-Theoretic Benchmarking of Species Tree Inference

This notebook presents a comparative workflow for species tree inference using two different character sets: traditional amino acid sequences and foldtree-encoded structural features. The goal is to benchmark the phylogenetic signal carried by each character set using information-theoretic approaches.

**Workflow Overview:**
- Construct a species tree using a standard amino acid-based pipeline (multiple sequence alignment, concatenation, and maximum likelihood inference).
- Construct an equivalent species tree using foldtree-encoded data.
- For both trees, compute column-wise log-likelihoods and character frequencies.
- Quantify the information content and phylogenetic signal of each character set by analyzing the distribution of likelihoods and character frequencies.

This approach enables a direct comparison of how much evolutionary information is captured by sequence versus structure-based encodings, providing an objective benchmark for future phylogenomic analyses.

In [1]:
cd /home/dmoi/projects/foldtree2/

/home/dmoi/projects/foldtree2


In [2]:
save_dir = './foldtree2/notebooks/benchmarks/Information_benchmark/'

In [3]:
from Bio import AlignIO
from Bio.Align import MultipleSeqAlignment
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import os
import requests
import random
import subprocess
from pathlib import Path
import concurrent.futures
import multiprocessing
import pandas as pd
import requests
from Bio import SeqIO
import re
import time

In [4]:
overwrite = False


In [5]:
import glob
markers = glob.glob( './families/Information_benchmark/marker_genes/marker_genes/*.fa')
print( f"Found {len(markers)} marker genes" )

Found 500 marker genes


In [6]:
#use autoreload to reload modules
%load_ext autoreload
%autoreload 2

In [8]:
# Specify your model and associated matrices
model = 'mergeddecoder_foldtree2_test'
from foldtree2 import ft2treebuilder as treebuilder
model_path = os.path.join( "models" , model )  # Path to your model (without .pkl)
mafftmat = model_path + "_mafftmat.mtx"
submat = model_path + "_submat.txt"
aapropcsv = os.path.join( 'foldtree2', "config" , "aaindex1.csv" )  # Path to your amino acid properties CSV file
# Initialize the treebuilder class
mafftmat = os.path.join(model_path + '_mafftmat.mtx')
submat = os.path.join(model_path + '_submat.txt')
charmaps = os.path.join(model_path + '_pair_counts.pkl')

tb = treebuilder.treebuilder(model=model_path, mafftmat=mafftmat, submat=submat , charmaps=charmaps, aapropcsv=aapropcsv ,  raxml_path='./foldtree2/raxml-ng/raxml-ng' ,
     maffttext2hex='./foldtree2/mafft_tools/maffttext2hex', maffthex2text='./foldtree2/mafft_tools/hex2maffttext')

#check that the paths are correct
assert os.path.exists(tb.mafftmat), f"MAFFT matrix not found at {tb.mafftmat}"
assert os.path.exists(tb.submat), f"Substitution matrix not found at {tb.submat}"
#check the tools

assert os.path.exists(tb.raxml_path), f"RAxML-NG not found at {tb.raxml_path}"
assert os.path.exists(tb.maffttext2hex), f"MAFFT text to hex converter not found at {tb.maffttext2hex}"
assert os.path.exists(tb.maffthex2text), f"MAFFT hex to text converter not found at {tb.maffthex2text}"

/home/dmoi/projects/foldtree2/foldtree2/ft2treebuilder.py:38: SyntaxWarning: invalid escape sequence '\ '
  self.raxmlchars = """0 1 2 3 4 5 6 7 8 9 A B C D E F G H I J K L M N O P Q R S T U V W X Y Z ! " # $ % & ' ( ) * + , / : ; < = > @ [ \ ] ^ _ { | } ~"""


loading charmaps from models/mergeddecoder_foldtree2_test_pair_counts.pkl


In [9]:
import subprocess
import os

def run_raxml_lg_alignment(alignment_file, output_prefix="raxml_lg_tree", raxml_path="raxml-ng"):
	"""
	Run RAxML-NG on an amino acid alignment using the LG+G+I model.

	Parameters:
	-----------
	alignment_file : str
		Path to the aligned FASTA file (amino acid sequences)
	output_prefix : str
		Prefix for RAxML-NG output files
	raxml_path : str
		Path to the RAxML-NG executable (default: 'raxml-ng')

	Returns:
	--------
	str: Path to the inferred tree file (Newick format)
	"""
	tree_file = f"{output_prefix}.raxml.bestTree"
	cmd = [
		raxml_path,
		"--msa", alignment_file,
		"--model", "LG+G+I",
		"--prefix", output_prefix,
		"--threads", "1"
	]
	print(f"Running: {' '.join(cmd)}")
	subprocess.run(cmd, check=True)
	if os.path.exists(tree_file):
		print(f"Inferred tree saved to {tree_file}")
		return tree_file
	else:
		print("RAxML-NG did not produce a tree file.")
		return None

In [10]:
def align_fasta_with_mafft(fasta_file):
	"""
	Align a single FASTA file using MAFFT
	
	Parameters:
	-----------
	fasta_file : str
		Path to input FASTA file
	
	Returns:
	--------
	tuple: (input_path, output_path, success_status)
	"""
	# Create output filename - same directory but with .aligned.fa extension
	input_path = Path(fasta_file)
	output_path = input_path.with_name(f"{input_path.stem}.aligned.fa")
	
	# Run MAFFT
	cmd = f"mafft --auto --thread 1 {fasta_file} > {output_path}"
	print(f"Aligning: {fasta_file}")
	
	try:
		subprocess.run(cmd, shell=True, check=True, stderr=subprocess.PIPE)
		return (fasta_file, str(output_path), True)
	except subprocess.CalledProcessError as e:
		print(f"Error aligning {fasta_file}: {e}")
		return (fasta_file, str(output_path), False)

# Get number of available cores (leave 1 core free for system processes)
max_workers = max(1, multiprocessing.cpu_count() - 1)
print(f"Using {max_workers} cores for alignments")

# Process alignments in parallel
aligned_files = []
failed_files = []


# Get number of available cores (leave 1 core free for system processes)
max_workers = max(1, multiprocessing.cpu_count() - 1)
print(f"Using {max_workers} cores for alignments")
# Process alignments in parallel
aligned_files = []
failed_files = []

Using 31 cores for alignments
Using 31 cores for alignments


In [11]:
def concatenate_alignments(alignment_files, output_file):
	"""
	Concatenate multiple alignment files into a single supermatrix alignment.
	
	Parameters:
	-----------
	alignment_files : list
		List of paths to alignment files in FASTA format
	output_file : str
		Path to save the concatenated alignment
		
	Returns:
	--------
	tuple
		(concatenated_alignment, partition_info)
		- concatenated_alignment: The final MultipleSeqAlignment object
		- partition_info: Dictionary with gene boundaries for partition file creation
	"""
	if not alignment_files:
		print("No alignment files provided")
		return None, {}
	
	# Dictionary to store sequences for each species across all genes
	all_species = {}
	partition_info = {}
	current_position = 1
	
	# Process each alignment file
	for i, aln_file in enumerate(alignment_files):
		try:
			# Load the alignment
			gene_name = Path(aln_file).stem.replace('.aligned', '')
			alignment = AlignIO.read(aln_file, "fasta")
			aln_length = alignment.get_alignment_length()
			
			# Store partition information
			partition_info[gene_name] = {
				'start': current_position,
				'end': current_position + aln_length - 1
			}
			
			# Process each sequence in this alignment
			for record in alignment:
				# Extract species identifier from the sequence header
				species_id = record.id.split('|')[-1]
				
				# Initialize this species entry if it doesn't exist yet
				if species_id not in all_species:
					all_species[species_id] = {}
				
				# Add this gene's sequence for this species
				all_species[species_id][gene_name] = str(record.seq)
			
			current_position += aln_length
			print(f"Processed alignment {i+1}/{len(alignment_files)}: {gene_name} ({aln_length} columns)")
			
		except Exception as e:
			print(f"Error processing {aln_file}: {str(e)}")
	
	# Create the concatenated alignment
	concatenated_records = []
	gene_names = list(partition_info.keys())
	
	for species_id, genes in all_species.items():
		# Build the concatenated sequence for this species
		concat_seq = ""
		for gene in gene_names:
			if gene in genes:
				concat_seq += genes[gene]
			else:
				# If this species doesn't have this gene, add gaps
				gene_length = partition_info[gene]['end'] - partition_info[gene]['start'] + 1
				concat_seq += "-" * gene_length
		
		# Create a SeqRecord for this concatenated sequence
		record = SeqRecord(
			Seq(concat_seq),
			id=species_id,
			description=f"Concatenated {len(gene_names)} genes"
		)
		concatenated_records.append(record)
	
	# Create and save the concatenated alignment
	concatenated_alignment = MultipleSeqAlignment(concatenated_records)
	
	# Save alignment to file
	with open(output_file, "w") as handle:
		AlignIO.write(concatenated_alignment, handle, "fasta")
	
	# Create a partition file for RAxML-NG
	partition_file = f"{output_file}.partition"
	with open(partition_file, "w") as handle:
		for gene, pos in partition_info.items():
			handle.write(f"GTR+G, {gene} = {pos['start']}-{pos['end']}\n")
	
	print(f"Created concatenated alignment with {len(concatenated_records)} species and {concatenated_alignment.get_alignment_length()} columns")
	print(f"Partition file saved to {partition_file}")
	return concatenated_alignment, partition_info

In [12]:
def extract_oma_ids_from_fasta(fasta_file):
	"""
	Extract OMA identifiers from a FASTA file
	ID format example:

	>MOUSE45461 | OMA754554 | COQ5_MOUSE | [Mus musculus]

	
	Parameters:
	-----------
	fasta_file : str
		Path to FASTA file
		
	Returns:
	--------
	list of str: OMA identifiers
	"""
	oma_ids = []
	oma_files = {}
	for record in SeqIO.parse(fasta_file, "fasta"):
		# Extract OMA ID from the FASTA header using the pipe-separated format
		id = record.description.split('|')[0]	
		oma_ids.append(id.strip())
	oma_files[id.strip()] = fasta_file
	return oma_ids , oma_files
# Extract OMA IDs from all marker files
oma_ids = []
oma_files = {}
for marker in markers:
	ids, files = extract_oma_ids_from_fasta(marker)
	oma_ids.extend(ids)
	oma_files.update(files)

oma_ids = list(set(oma_ids))  # Remove duplicates
print(f"Extracted {len(oma_ids)} unique OMA IDs from marker genes")
oma_files = pd.DataFrame.from_dict(oma_files, orient='index', columns=['fasta_file'])
oma_files.index.name = 'oma_id'
oma_files.reset_index(inplace=True)
oma_files.to_csv("oma_markergene_files.csv", index=False)


Extracted 8377 unique OMA IDs from marker genes


In [13]:
print( oma_ids[:10] )  # Print first 10 OMA IDs for verification

['NAEGR13256', 'YEAST02461', 'HUMAN102995', 'NAEGR05161', 'ACACA06967', 'HUMAN03132', 'ARATH17090', 'PARTE06860', 'CAPO306280', 'DICDI11709']


In [14]:
import json
import tqdm
def bulk_map_oma_to_uniprot(oma_ids, batch_size=100, retry_limit=3 , verbose=True):
	"""
	Map a list of OMA identifiers to UniProt using the OMA bulk API.

	Parameters:
	-----------
	oma_ids : list of str
		List of OMA identifiers (e.g., OMA123456)
	batch_size : int
		Number of IDs per batch (max 1000)
	retry_limit : int
		Number of times to retry on failure

	Returns:
	--------
	list of dict: Mapping information for each OMA ID
	"""
	url = "https://omabrowser.org/api/protein/bulk_retrieve/"
	all_results = []

	for i in tqdm.tqdm(range(0, len(oma_ids), batch_size)):
		batch = oma_ids[i:i+batch_size]
		for attempt in range(retry_limit):
			try:
				response = requests.post(url, json={"ids": batch})
				if response.ok:
					data = response.json()
					data = json.loads(response.text)  # Ensure we parse the JSON correctly
					#transfor data into dataframe
					results = {}
					for query in data:
						results[query['query_id']] = query['target']
					if verbose:
						print(f"Batch {i//batch_size+1} processed successfully with {len(results)} mappings")
						print(pd.DataFrame.from_dict(results, orient='index').reset_index())
					all_results.append(pd.DataFrame.from_dict(results, orient='index').reset_index())
					break  # Success, break retry loop
				else:
					print(f"Error fetching batch {i//batch_size+1}: HTTP {response.status_code}")
			except Exception as e:
				print(f"Error in batch {i//batch_size+1}: {str(e)}")
			time.sleep(1)
		else:
			# If all retries failed, add minimal info for each ID in the batch
			for oma_id in batch:
				all_results.append({'oma_id': oma_id, 'uniprot_id': None})
			all_results.append(pd.DataFrame([{'oma_id': oma_id, 'uniprot_id': None}]))
	# Concatenate all results into a single DataFrame
	if all_results:
		all_results = pd.concat(all_results, ignore_index=True)
		return all_results
	else:
		return pd.DataFrame(columns=['oma_id', 'uniprot_id'])


	

In [15]:
import os
# Check if the mapping file already exists
if not os.path.exists(save_dir):
	print(f"Creating directory {save_dir} for marker gene files...")
	os.makedirs(save_dir, exist_ok=True)


In [16]:
# Check if the mapping file already exists
mapping_file = os.path.join(save_dir, "oma_to_uniprot_mapping.csv")
if not os.path.exists(mapping_file):
	print("Mapping file does not exist, creating it now...")
	#retreive all OMA IDs from the marker genes
	mapping_df = bulk_map_oma_to_uniprot(oma_ids , verbose=False)
	# Save the mapping results to a CSV file
	mapping_df.to_csv(mapping_file, index=True)
else:
	print("Mapping file already exists, loading it...")
	mapping_df = pd.read_csv(mapping_file, index_col=0)
	print(f"Loaded mapping file with {len(mapping_df)} entries")
	print(mapping_df.head())
	

Mapping file already exists, loading it...
Loaded mapping file with 8377 entries
        index  entry_nr                                     entry_url  \
0  TRYB205898   7201087   https://omabrowser.org/api/protein/7201087/   
1  DROME24395  17556944  https://omabrowser.org/api/protein/17556944/   
2  YEAST00066   8927628   https://omabrowser.org/api/protein/8927628/   
3  THAPS06704  20296723  https://omabrowser.org/api/protein/20296723/   
4  DROME17519  17550068  https://omabrowser.org/api/protein/17550068/   

        omaid  canonicalid                      sequence_md5  sequence_length  \
0  TRYB205898       Q57ZU2  c077db9e5ee8aa5e763b0daf676859b6              860   
1  DROME24395   GATA_DROME  6f214dfc8f1dbf0393dd24e6aac8ea1a              508   
2  YEAST00066  HSP71_YEAST  7cdc576cd47040f8a215326a8e29e066              642   
3  THAPS06704       B8BYD7  c493f1360799e587e42ffc17c5f33a57             1103   
4  DROME17519       Q9VVI0  4827054d86ce99be2f09ad82c7666978              5

In [17]:

import pyoma.browser.db
db = pyoma.browser.db.Database("/home/dmoi/datasets/OMA/OmaServer.h5")
db.get_release_name()
resolver = pyoma.browser.db.IDResolver(db)
mapper = db.id_mapper['OMA']
linkout = db.id_mapper['Linkout'] 




Cannot load SequenceSearch. Any future call to seq_search will fail!
Traceback (most recent call last):
  File "/home/dmoi/miniforge3/envs/pyg/lib/python3.12/site-packages/pyoma/browser/db.py", line 2340, in __init__
    self.seq_idx = self.seq_idx()
                   ^^^^^^^^^^^^^^
  File "/home/dmoi/miniforge3/envs/pyg/lib/python3.12/site-packages/tables/link.py", line 427, in __call__
    self.extfile = tb.open_file(filename, **kwargs)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/dmoi/miniforge3/envs/pyg/lib/python3.12/site-packages/pyoma/browser/db.py", line 137, in synchronized_open_file
    return _tables_file._original_open_file(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/dmoi/miniforge3/envs/pyg/lib/python3.12/site-packages/tables/file.py", line 325, in open_file
    return File(filename, mode, title, root_uep, filters, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/hom

In [18]:

entry = 'CHLRE00910'  # Example entry number
entry_nr = mapper.omaid_to_entry_nr(entry)
print(f"Entry number for {entry}: {entry_nr}")


Entry number for CHLRE00910: 20577393


In [19]:
#try mapping entry
entry = 'CHLRE00910'  # Example entry number
entry_nr = mapper.omaid_to_entry_nr(entry)
xrefs = linkout.iter_xrefs_for_entry_nr(entry_nr)
print([ r for r in xrefs])


[{'source': 'UniProtKB/TrEMBL', 'xref': 'A0A2K3DKT0', 'url': 'http://uniprot.org/uniprot/A0A2K3DKT0'}, {'source': 'EntrezGene', 'xref': '5718125', 'url': 'http://www.ncbi.nlm.nih.gov/gene/5718125'}]


In [20]:
def get_uniprot_ids_from_oma(oma_id, mapper, linkout):
	"""
	Get UniProt IDs for a given OMA ID by chaining mapper and xref operations.
	
	Parameters:
	-----------
	oma_id : str
		OMA identifier (e.g., 'CHLRE00910')
	mapper : pyoma.browser.db.OmaIdMapper
		OMA ID mapper instance
	linkout : pyoma.browser.idmapper.LinkoutIdMapper
		Linkout mapper instance
	
	Returns:
	--------
	list: UniProt IDs associated with the OMA ID
	"""
	try:
		# Convert OMA ID to entry number
		entry_nr = mapper.omaid_to_entry_nr(oma_id)
		
		# Get all xrefs for this entry
		xrefs = linkout.iter_xrefs_for_entry_nr(entry_nr)
		
		# Filter for UniProt references
		uniprot_ids = [ref['xref'] for ref in xrefs 
					  if ref['source'] in ['UniProtKB/Swiss-Prot', 'UniProtKB/TrEMBL']]
		
		return uniprot_ids
	
	except Exception as e:
		print(f"Error processing {oma_id}: {str(e)}")
		return []

# Test the function with the example entry
uniprot_ids = get_uniprot_ids_from_oma(entry, mapper, linkout)
print(f"UniProt IDs for {entry}: {uniprot_ids}")

UniProt IDs for CHLRE00910: ['A0A2K3DKT0']


In [21]:
print( mapping_df.head() )

        index  entry_nr                                     entry_url  \
0  TRYB205898   7201087   https://omabrowser.org/api/protein/7201087/   
1  DROME24395  17556944  https://omabrowser.org/api/protein/17556944/   
2  YEAST00066   8927628   https://omabrowser.org/api/protein/8927628/   
3  THAPS06704  20296723  https://omabrowser.org/api/protein/20296723/   
4  DROME17519  17550068  https://omabrowser.org/api/protein/17550068/   

        omaid  canonicalid                      sequence_md5  sequence_length  \
0  TRYB205898       Q57ZU2  c077db9e5ee8aa5e763b0daf676859b6              860   
1  DROME24395   GATA_DROME  6f214dfc8f1dbf0393dd24e6aac8ea1a              508   
2  YEAST00066  HSP71_YEAST  7cdc576cd47040f8a215326a8e29e066              642   
3  THAPS06704       B8BYD7  c493f1360799e587e42ffc17c5f33a57             1103   
4  DROME17519       Q9VVI0  4827054d86ce99be2f09ad82c7666978              553   

                                             species  oma_group  \
0  {'co

In [22]:
def ret_refs(oma_ids, mapper, linkout):
	"""
	Retrieve UniProt references for given OMA IDs from the OMA database.
	
	Parameters:
	-----------
	oma_ids : list
		List of OMA identifiers
	mapper : pyoma.browser.db.OmaIdMapper
		OMA ID mapper instance
	linkout : pyoma.browser.idmapper.LinkoutIdMapper
		Linkout mapper instance
	
	Returns:
	--------
	dict: Dictionary mapping OMA IDs to their UniProt references
	"""
	return {oma_id: get_uniprot_ids_from_oma(oma_id, mapper, linkout) for oma_id in oma_ids}

# Try with some OMA IDs
sample_oma_ids = mapping_df['index'].sample(10).tolist()
print(f"Sample OMA IDs: {sample_oma_ids}")

sample_refs = ret_refs(sample_oma_ids, mapper, linkout)
print("\nSample references:")
for oma_id, refs in sample_refs.items():
	print(f"{oma_id}: {refs}")

Sample OMA IDs: ['CHLRE16464', 'CAPO305630', 'MOUSE31178', 'DROME09011', 'HUMAN52035', 'HUMAN21192', 'ARATH15907', 'PARTE35951', 'CYAME04251', 'CYAME01940']

Sample references:
CHLRE16464: ['A8JHC3']
CAPO305630: ['A0A0D2WMH7']
MOUSE31178: ['Q60737', 'Q61177', 'Q6NSS6']
DROME09011: ['Q960Q8']
HUMAN52035: ['A6PVX3', 'P55036']
HUMAN21192: ['A0A024R6K8', 'G3V227', 'G3V277', 'G3V2C0', 'G3V2F2', 'G3V313', 'G3V339', 'G3V3H8', 'G3V3P2', 'G3V3R3', 'G3V3S7', 'G3V3X0', 'G3V3Y5', 'G3V423', 'G3V456', 'G3V4C7', 'G3V4N8', 'G3V4S4', 'G3V5H5', 'G3V5U1', 'G3V5W1', 'P23381', 'P78534']
ARATH15907: ['A0A178VTX1', 'Q8LPJ7']
PARTE35951: ['A0DY77']
CYAME04251: []
CYAME01940: []


In [23]:
from collections import defaultdict
import pandas as pd

def parse_xref_list_to_row(xref_list):
	"""
	Parse a list of xref dicts into a single DataFrame row.
	Each xref source becomes a column, value is the xref (or list if multiple for same source).
	The row index is the OMA ID (omaid).
	
	Parameters:
	-----------
	xref_list : list of dict
		List of xref dictionaries as returned by the OMA API.
		
	Returns:
	--------
	pandas.DataFrame: Single-row DataFrame with omaid as index and xrefs as columns.
	"""

	if not xref_list:
		return pd.DataFrame()

	omaid = xref_list[0].get('omaid')
	xref_dict = defaultdict(list)
	for x in xref_list:
		src = x['source']
		xref_dict[src].append(x['xref'])

	# Flatten single-item lists
	xref_dict = {k: v[0] if len(v) == 1 else v for k, v in xref_dict.items()}
	xref_dict['omaid'] = omaid

	df = pd.DataFrame([xref_dict]).set_index('omaid')
	return df

In [24]:
mapping_path = os.path.join(save_dir, "oma_to_uniprot_mapping.csv")
mapping_df = pd.read_csv(mapping_path, index_col=0)
example = mapping_df.iloc[0]
print( example)
#use ret_refs to get xref
#check xref pickle
xref_path = os.path.join(save_dir, "xref_data.pkl")
# Check if the xref data pickle file exists
if os.path.exists(xref_path):
	print("Loading xref data from pickle file...")
	xref_data = pd.read_pickle(xref_path)
else:
	print("Xref data pickle not found, generating it...")
	# Generate xref data
	# This will take some time depending on the number of OMA IDs
	# and the speed of the OMA API
	xref_data = ret_refs(mapping_df.omaid.to_list() , mapper, linkout)
#map xref_data to a DataFrame
print(xref_data)

index                                                               TRYB205898
entry_nr                                                               7201087
entry_url                          https://omabrowser.org/api/protein/7201087/
omaid                                                               TRYB205898
canonicalid                                                             Q57ZU2
sequence_md5                                  c077db9e5ee8aa5e763b0daf676859b6
sequence_length                                                            860
species                      {'code': 'TRYB2', 'taxon_id': 185431, 'species...
oma_group                                                              1428576
oma_hog_id                                                     HOG:E0802393.2b
chromosome                                                                   5
locus                            {'start': 932578, 'end': 935160, 'strand': 1}
is_main_isoform                                     

In [25]:
mapping_df['uniprot_refs'] = mapping_df['omaid'].map(xref_data)

In [26]:
#save with uniprot_refs
map_out = os.path.join(save_dir, "oma_to_uniprot_mapping_with_refs.csv")
mapping_df.to_csv(map_out, index=False)

In [27]:
print( mapping_df.head() )

        index  entry_nr                                     entry_url  \
0  TRYB205898   7201087   https://omabrowser.org/api/protein/7201087/   
1  DROME24395  17556944  https://omabrowser.org/api/protein/17556944/   
2  YEAST00066   8927628   https://omabrowser.org/api/protein/8927628/   
3  THAPS06704  20296723  https://omabrowser.org/api/protein/20296723/   
4  DROME17519  17550068  https://omabrowser.org/api/protein/17550068/   

        omaid  canonicalid                      sequence_md5  sequence_length  \
0  TRYB205898       Q57ZU2  c077db9e5ee8aa5e763b0daf676859b6              860   
1  DROME24395   GATA_DROME  6f214dfc8f1dbf0393dd24e6aac8ea1a              508   
2  YEAST00066  HSP71_YEAST  7cdc576cd47040f8a215326a8e29e066              642   
3  THAPS06704       B8BYD7  c493f1360799e587e42ffc17c5f33a57             1103   
4  DROME17519       Q9VVI0  4827054d86ce99be2f09ad82c7666978              553   

                                             species  oma_group  \
0  {'co

In [28]:
mapping_df['1stref'] = mapping_df['uniprot_refs'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None)
#eliminate rows with no 1stref
mapping_df = mapping_df[mapping_df['1stref'].notnull()]
print(mapping_df.head())

        index  entry_nr                                     entry_url  \
0  TRYB205898   7201087   https://omabrowser.org/api/protein/7201087/   
1  DROME24395  17556944  https://omabrowser.org/api/protein/17556944/   
2  YEAST00066   8927628   https://omabrowser.org/api/protein/8927628/   
3  THAPS06704  20296723  https://omabrowser.org/api/protein/20296723/   
4  DROME17519  17550068  https://omabrowser.org/api/protein/17550068/   

        omaid  canonicalid                      sequence_md5  sequence_length  \
0  TRYB205898       Q57ZU2  c077db9e5ee8aa5e763b0daf676859b6              860   
1  DROME24395   GATA_DROME  6f214dfc8f1dbf0393dd24e6aac8ea1a              508   
2  YEAST00066  HSP71_YEAST  7cdc576cd47040f8a215326a8e29e066              642   
3  THAPS06704       B8BYD7  c493f1360799e587e42ffc17c5f33a57             1103   
4  DROME17519       Q9VVI0  4827054d86ce99be2f09ad82c7666978              553   

                                             species  oma_group  \
0  {'co

In [29]:
mapping_df['struct_folder'] = mapping_df['oma_group'].apply(lambda x: f"./families/Information_benchmark/marker_genes/{x}/structs")

In [30]:
from foldtree2.src.AFDB_tools import grab_struct
import tqdm

In [32]:
dl = True
if dl:
	#download the alphafold structures for each marker gene family
	for idx,row in tqdm.tqdm(mapping_df.iterrows()):
		#check if the folder exists
		struct_folder = row['struct_folder']
		if not os.path.exists(struct_folder):
			os.makedirs(struct_folder)
		#download the structure
		retpath = grab_struct(row['1stref'], struct_folder, overwrite=overwrite)
		#add struct path to mapping_df
		mapping_df.at[idx, 'struct_path'] = retpath
	#save the mapping_df with struct paths
	map_out = os.path.join(save_dir, "oma_to_uniprot_mapping_with_structs.csv")
	mapping_df.to_csv(map_out, index=False)
if not dl:
	#load the mapping_df with struct paths
	map_out = os.path.join(save_dir, "oma_to_uniprot_mapping_with_structs.csv")

15it [00:00, 77.87it/s]

error downloading structure for A0A8H4BYN2 HTTP Error 404: Not Found
https://alphafold.ebi.ac.uk/files/AF-A0A8H4BYN2-F1-model_v4.pdb
error downloading structure for A0A2K3DAU6 HTTP Error 404: Not Found
https://alphafold.ebi.ac.uk/files/AF-A0A2K3DAU6-F1-model_v4.pdb
error downloading structure for O15881 HTTP Error 404: Not Found
https://alphafold.ebi.ac.uk/files/AF-O15881-F1-model_v4.pdb


182it [00:00, 431.40it/s]

error downloading structure for L8HD42 HTTP Error 404: Not Found
https://alphafold.ebi.ac.uk/files/AF-L8HD42-F1-model_v4.pdb
error downloading structure for X6P1L5 HTTP Error 404: Not Found
https://alphafold.ebi.ac.uk/files/AF-X6P1L5-F1-model_v4.pdb
error downloading structure for A0A1X7UM21 HTTP Error 404: Not Found
https://alphafold.ebi.ac.uk/files/AF-A0A1X7UM21-F1-model_v4.pdb


228it [00:00, 326.26it/s]

...

error downloading structure for A0A178W1F6 HTTP Error 404: Not Found
https://alphafold.ebi.ac.uk/files/AF-A0A178W1F6-F1-model_v4.pdb
error downloading structure for D2VJC0 HTTP Error 404: Not Found
https://alphafold.ebi.ac.uk/files/AF-D2VJC0-F1-model_v4.pdb
error downloading structure for A0D8K9 HTTP Error 404: Not Found
https://alphafold.ebi.ac.uk/files/AF-A0D8K9-F1-model_v4.pdb


7556it [00:17, 381.15it/s]

error downloading structure for A0A2K1L907 HTTP Error 404: Not Found
https://alphafold.ebi.ac.uk/files/AF-A0A2K1L907-F1-model_v4.pdb
error downloading structure for A0A8Q3SHT1 HTTP Error 404: Not Found
https://alphafold.ebi.ac.uk/files/AF-A0A8Q3SHT1-F1-model_v4.pdb
error downloading structure for A8K6F0 HTTP Error 404: Not Found
https://alphafold.ebi.ac.uk/files/AF-A8K6F0-F1-model_v4.pdb


7751it [00:17, 447.41it/s]

error downloading structure for A0A8H8UMD1 HTTP Error 404: Not Found
https://alphafold.ebi.ac.uk/files/AF-A0A8H8UMD1-F1-model_v4.pdb
error downloading structure for D2V0Z5 HTTP Error 404: Not Found
https://alphafold.ebi.ac.uk/files/AF-D2V0Z5-F1-model_v4.pdb
error downloading structure for A0A0D2X192 HTTP Error 404: Not Found
https://alphafold.ebi.ac.uk/files/AF-A0A0D2X192-F1-model_v4.pdb


In [33]:
#drop the rows with no struct_path
mapping_df = mapping_df[mapping_df['struct_path'].notnull()]
print(f"Final mapping DataFrame with structures: {mapping_df.shape[0]} entries")

Final mapping DataFrame with structures: 7521 entries


In [31]:
#write an identifier file for each marker gene family
for sub in mapping_df['oma_group'].unique():
	sub_df = mapping_df[mapping_df['oma_group'] == sub]
	#write the uniprot ids to an identifiers.txt file
	identifiers_file = f"./families/Information_benchmark/marker_genes/{sub}/identifiers.txt"
	with open(identifiers_file, 'w') as f:
		for uniprot_id in sub_df['1stref'].unique():
			f.write(f"{uniprot_id}\n")

In [32]:
from Bio import PDB

def create_fasta_from_pdbs(pdb_folder, output_fasta, chain_id='A', species_mapping=None):
	"""
	Create a FASTA file from PDB structures by extracting amino acid sequences using Biopython.
	
	Parameters:
	-----------
	pdb_folder : str
		Path to folder containing PDB files
	output_fasta : str
		Path to output FASTA file
	chain_id : str, optional
		Chain ID to extract sequence from (default: 'A')
	species_mapping : dict, optional
		Dictionary mapping PDB IDs to species codes
		
	Returns:
	--------
	list: List of SeqRecord objects that were written to FASTA
	"""
	
	# Standard amino acid 3-letter to 1-letter conversion
	aa_codes = {
		'ALA': 'A', 'CYS': 'C', 'ASP': 'D', 'GLU': 'E',
		'PHE': 'F', 'GLY': 'G', 'HIS': 'H', 'ILE': 'I',
		'LYS': 'K', 'LEU': 'L', 'MET': 'M', 'ASN': 'N',
		'PRO': 'P', 'GLN': 'Q', 'ARG': 'R', 'SER': 'S',
		'THR': 'T', 'VAL': 'V', 'TRP': 'W', 'TYR': 'Y'
	}
	
	records = []
	parser = PDB.PDBParser(QUIET=True)
	
	# Find all PDB files in the folder
	pdb_files = glob.glob(os.path.join(pdb_folder, "*.pdb"))
	
	for pdb_file in pdb_files:
		try:
			# Get structure name from filename
			pdb_id = os.path.basename(pdb_file).replace('.pdb', '')
			
			# Parse structure
			structure = parser.get_structure(pdb_id, pdb_file)
			
			# Get first model
			model = structure[0]
			
			# Check if requested chain exists
			if chain_id in model:
				chain = model[chain_id]
				
				# Extract sequence
				sequence = ''
				for residue in chain:
					if residue.get_resname() in aa_codes:
						sequence += aa_codes[residue.get_resname()]
				
				if sequence:
					# Create record ID with species code if available
					if species_mapping and pdb_id in species_mapping:
						record_id = f"{pdb_id}|{species_mapping[pdb_id]}"
						description = f"Chain {chain_id} sequence from {pdb_file} - Species: {species_mapping[pdb_id]}"
					else:
						record_id = pdb_id
						description = f"Chain {chain_id} sequence from {pdb_file}"
					
					# Create SeqRecord object
					record = SeqRecord(
						Seq(sequence),
						id=record_id,
						description=description
					)
					records.append(record)
			
		except Exception as e:
			print(f"Error processing {pdb_file}: {str(e)}")
	
	# Write all sequences to FASTA file
	if records:
		with open(output_fasta, 'w') as handle:
			SeqIO.write(records, handle, 'fasta')
		print(f"Created FASTA file with {len(records)} sequences at {output_fasta}")
	else:
		print("No sequences were extracted from PDB files")
	
	return records

In [33]:
#create a FASTA file for each marker gene family and use the species mapping from the sub datafram

for sub in mapping_df['oma_group'].unique():
	sub_df = mapping_df[mapping_df['oma_group'] == sub]
	struct_folder = sub_df['struct_folder'].iloc[0]  # Use the first struct folder for this family
	output_fasta = f"./families/Information_benchmark/marker_genes/{sub}/sequences.fasta"
	
	# Create species mapping for this family
	species_mapping = dict( (zip(sub_df['1stref'], sub_df['omaid'].apply(lambda x: x[:5]))) )
	if not os.path.exists(output_fasta):
		print(f"Creating FASTA file for {sub} in {output_fasta}")
		create_fasta_from_pdbs(struct_folder, output_fasta, chain_id='A', species_mapping=species_mapping)
	else:
		print(f"FASTA file already exists for {sub}, skipping creation")

FASTA file already exists for 1428576, skipping creation
FASTA file already exists for 966408, skipping creation
FASTA file already exists for 1067505, skipping creation
FASTA file already exists for 1314601, skipping creation
FASTA file already exists for 1067338, skipping creation
FASTA file already exists for 848449, skipping creation
FASTA file already exists for 1392015, skipping creation
FASTA file already exists for 756269, skipping creation
FASTA file already exists for 1344491, skipping creation
FASTA file already exists for 1372179, skipping creation
FASTA file already exists for 1067608, skipping creation
FASTA file already exists for 1324983, skipping creation
FASTA file already exists for 1427916, skipping creation
FASTA file already exists for 1067292, skipping creation
FASTA file already exists for 1384924, skipping creation
FASTA file already exists for 770664, skipping creation
FASTA file already exists for 1396383, skipping creation
FASTA file already exists for 10132

In [34]:
 #use normal mafft to align the sequences
import glob
from concurrent.futures import ThreadPoolExecutor
aln = False
if aln:
	# Get all FASTA files in the marker gene families
	fasta_files = glob.glob("./families/Information_benchmark/marker_genes/*/sequences.fasta")
	print(f"Found {len(fasta_files)} FASTA files for alignment")
	# Align all FASTA files in parallel
	max_workers = 10  # Leave 1 core free for system processes

	with ThreadPoolExecutor(max_workers=max_workers) as executor:
		results = list(executor.map(align_fasta_with_mafft, tqdm.tqdm(fasta_files)))

In [36]:
#for each marker gene, encode the structs, align and make a tree using treebuilder
import foldtree2 as ft2

for fam in tqdm.tqdm(mapping_df['oma_group'].unique()):
	fam_df = mapping_df[mapping_df['oma_group'] == fam]
	#check if the folder exists
	struct_folder = fam_df['struct_folder'].iloc[0]  # Use the first struct folder for this family
	#check if the aligned file exists
	outfile = f"./families/Information_benchmark/marker_genes/{fam}/encoded.ASCIIaln.txt.raxml.bestTree"
	outdir = os.path.dirname(outfile)
	if not os.path.exists(outfile) or overwrite:
		outfasta = os.path.join(outdir, 'encoded.fasta')
		print(f"Encoding structures for {fam} into {outfasta}")
		
		try:	
			results = tb.structs2tree( structs=struct_folder+'/*.pdb' , outdir=outdir, overwrite=overwrite, mafftmat=mafftmat, submat=submat, aapropcsv=aapropcsv )
			#results is a dictionary with keys 'encoded_fasta', 'tree', 'alignment', 'mafft_aln', 'asciifile', 'hexfasta'
			encoded_fasta = results['encoded_fasta']
			treefile = results['tree']
			alnfasta = results['alignment']
			mafftaln = results['mafft_aln']
			asciifile = results['asciifile']
			hexfasta = results['hexfasta']
			print(f"Encoded FASTA: {encoded_fasta}")
			print(f"Tree file: {treefile}")
			print(f"Alignment FASTA: {alnfasta}")
			print(f"MAFFT alignment: {mafftaln}")
			print(f"ASCII file: {asciifile}")
			print(f"Hex FASTA: {hexfasta}")
		except Exception as e:
			print(f"Error processing {fam}: {str(e)}")
			continue
	else:
		print(f"Output file {outfile} already exists, skipping encoding for {fam}")

 86%|▊| 428/500 [00

Output file ./families/Information_benchmark/marker_genes/1428576/encoded.ASCIIaln.txt.raxml.bestTree already exists, skipping encoding for 1428576
Output file ./families/Information_benchmark/marker_genes/966408/encoded.ASCIIaln.txt.raxml.bestTree already exists, skipping encoding for 966408
Output file ./families/Information_benchmark/marker_genes/1067505/encoded.ASCIIaln.txt.raxml.bestTree already exists, skipping encoding for 1067505
Output file ./families/Information_benchmark/marker_genes/1314601/encoded.ASCIIaln.txt.raxml.bestTree already exists, skipping encoding for 1314601
Output file ./families/Information_benchmark/marker_genes/1067338/encoded.ASCIIaln.txt.raxml.bestTree already exists, skipping encoding for 1067338
Output file ./families/Information_benchmark/marker_genes/848449/encoded.ASCIIaln.txt.raxml.bestTree already exists, skipping encoding for 848449
Output file ./families/Information_benchmark/marker_genes/1392015/encoded.ASCIIaln.txt.raxml.bestTree already exists

converting structures



 86%|▊| 428/500 [00


100%|█| 3/3 [00:56<
Encoding structures to FASTA: 3it [00:56, 18.89s/it]


converting to hex for mafft
outfile for hex : ./families/Information_benchmark/marker_genes/768489/encoded.hex
converting to ascii for mafft
outfile for ascii : ./families/Information_benchmark/marker_genes/768489/encoded.ASCII
./foldtree2/mafft_tools/hex2maffttext ./families/Information_benchmark/marker_genes/768489/encoded.hex > ./families/Information_benchmark/marker_genes/768489/encoded.ASCII
asciifile: ./families/Information_benchmark/marker_genes/768489/encoded.ASCII
running mafft
mafft --text --thread -1 --localpair --maxiterate 1000 --textmatrix models/mergeddecoder_foldtree2_test_mafftmat.mtx ./families/Information_benchmark/marker_genes/768489/encoded.ASCII  > ./families/Information_benchmark/marker_genes/768489/encoded.ASCIIaln.txt


...

converting to hex for mafft
outfile for hex : ./families/Information_benchmark/marker_genes/1424224/encoded.hex
converting to ascii for mafft
outfile for ascii : ./families/Information_benchmark/marker_genes/1424224/encoded.ASCII
./foldtree2/mafft_tools/hex2maffttext ./families/Information_benchmark/marker_genes/1424224/encoded.hex > ./families/Information_benchmark/marker_genes/1424224/encoded.ASCII
asciifile: ./families/Information_benchmark/marker_genes/1424224/encoded.ASCII
running mafft
mafft --text --thread -1 --localpair --maxiterate 1000 --textmatrix models/mergeddecoder_foldtree2_test_mafftmat.mtx ./families/Information_benchmark/marker_genes/1424224/encoded.ASCII  > ./families/Information_benchmark/marker_genes/1424224/encoded.ASCIIaln.txt


OS = linux
The number of physical cores =  16
outputhat23=16
treein = 0
compacttree = 0
stacksize: 8192 kb
nalphabets = 256
nused=
All-to-all alignment.
tbfast-pair (text) Version 7.526
alg=L, model=Extended, 2.00, -0.10, +0.10, noshift, amax=0.0
16 thread(s)

outputhat23=16
Loading 'hat3.seed' ... 
done.
Writing hat3 for iterative refinement
nalphabets = 256
nused=
Gap Penalty = -1.53, +0.00, +0.00
tbutree = 1, compacttree = 0
Constructing a UPGMA tree ... 
    0 / 5
done.

Progressive alignment ... 
STEP     2 /4 (thread    0) 
Reallocating (by thread 3) ..done. *alloclen = 4547
STEP     4 /4 (thread    2) 
done.
tbfast (text) Version 7.526
alg=A, model=Extended, 1.53, -0.00, -0.00, noshift, amax=0.0
16 thread(s)

minimumweight = 0.000010
autosubalignment = 0.000000
nthread = 8
randomseed = 0
blosum -2 / kimura 200
poffset = 0
niter = 16
sueff_global = 0.100000
nadd = 16
Loading 'hat3' ... done.
nalphabets = 256
nused=

    0 / 5
Segment   1/  1    1-1932
004-0006-1 (thread    3) wor

converting mafft aln to hex fasta
./foldtree2/mafft_tools/maffttext2hex ./families/Information_benchmark/marker_genes/1424224/encoded.ASCIIaln.txt > ./families/Information_benchmark/marker_genes/1424224/encoded.ASCIIaln.txt.hex
running raxml-ng
./foldtree2/raxml-ng/raxml-ng --model MULTI22_GTR{models/mergeddecoder_foldtree2_test_submat.txt} --redo  --all --bs-trees 20 --seed 12345 --threads auto{32} --workers auto --msa ./families/Information_benchmark/marker_genes/1424224/encoded.ASCIIaln.txt.raxml_aln.fasta --prefix ./families/Information_benchmark/marker_genes/1424224/encoded.ASCIIaln.txt --force perf_threads

RAxML-NG v. 1.2.2-master released on 30.04.2024 by The Exelixis Lab.
Developed by: Alexey M. Kozlov and Alexandros Stamatakis.
Contributors: Diego Darriba, Tomas Flouri, Benoit Morel, Sarah Lutteropp, Ben Bettisworth, Julia Haag, Anastasis Togkousidis.
Latest version: https://github.com/amkozlov/raxml-ng
Questions/problems/suggestions? Please visit: https://groups.google.com/f

100%|█| 500/500 [02

[00:00:01] [worker #3] Bootstrap tree #20, logLikelihood: -19469.831330
[00:00:01] [worker #0] Bootstrap tree #17, logLikelihood: -19470.504756
[00:00:01] [worker #2] Bootstrap tree #19, logLikelihood: -19459.080079
[00:00:01] [worker #1] Bootstrap tree #18, logLikelihood: -19191.280608

Optimized model parameters:

   Partition 0: noname
   Rate heterogeneity: NONE
   Base frequencies (user): 0.022373 0.014977 0.037758 0.027621 0.000018 0.065800 0.050222 0.040233 0.055168 0.042505 0.052079 0.079477 0.015546 0.165890 0.025259 0.072871 0.059498 0.017111 0.020838 0.065841 0.012980 0.055936 
   Substitution rates (user): 0.066803 0.068463 0.114533 0.134028 0.052453 0.044379 0.077626 0.061272 0.057495 0.047960 0.053751 0.069987 0.036514 0.088754 0.043550 0.072451 0.075666 0.059274 0.062504 0.069100 0.062370 0.068490 0.066493 0.040043 0.047035 0.065493 0.043314 0.042310 0.053859 0.041257 0.055141 0.074390 0.051561 0.048722 0.070380 0.045608 0.042372 0.070640 0.034222 0.072247 0.037389 0.136

In [38]:
import glob
import os
from Bio import AlignIO
from Bio.Align import MultipleSeqAlignment
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

In [39]:
#glob up all the sequence alignments
alignment_files = glob.glob("./families/Information_benchmark/marker_genes/*/aligned.fasta")
#glob up all the structural alignments
structural_alignment_files = glob.glob("./families/Information_benchmark/marker_genes/*/encoded.ASCIIaln.txt.raxml_aln.fasta")



In [ ]:
#create a tree for each marker gene using raxml and mafft in normal mode

for fam in tqdm.tqdm(mapping_df['oma_group'].unique()):
	fam_df = mapping_df[mapping_df['oma_group'] == fam]
	#check if the folder exists
	#check if the raxml tree already exists
	raxml_tree_file = f"./families/Information_benchmark/marker_genes/{fam}/raxml_lg_tree.raxml.bestTree"
	if not os.path.exists( f"./families/Information_benchmark/marker_genes/{fam}/raxml" ):
		struct_folder = fam_df['struct_folder'].iloc[0]  # Use the first struct folder for this family
		#pdb fasta 
		output_fasta = f"./families/Information_benchmark/marker_genes/{fam}/sequences.fasta"
		alnfast = align_fasta_with_mafft(output_fasta)[1]  # Get the aligned FASTA file path
		if alnfast is None:
			print(f"Alignment failed for {output_fasta}, skipping tree creation for {fam}")
			continue
		
		print(f"Creating RAxML tree for {fam} using aligned FASTA: {alnfast}")
		# Run RAxML-LG alignment and tree inference
		# Use the output prefix to save the tree file
		raxml_ng_path = "./foldtree2/raxml-ng/raxml-ng"
		tree = run_raxml_lg_alignment(alnfast, output_prefix=f"./families/Information_benchmark/marker_genes/{fam}/raxml_lg_tree", raxml_path=raxml_ng_path)

  0%|                                                                     | 0/500 [00:00<?, ?it/s]

Aligning: ./families/Information_benchmark/marker_genes/1428576/sequences.fasta
Creating RAxML tree for 1428576 using aligned FASTA: families/Information_benchmark/marker_genes/1428576/sequences.aligned.fa
Running: ./foldtree2/raxml-ng/raxml-ng --msa families/Information_benchmark/marker_genes/1428576/sequences.aligned.fa --model LG+G+I --prefix ./families/Information_benchmark/marker_genes/1428576/raxml_lg_tree --threads 1

RAxML-NG v. 1.2.2-master released on 30.04.2024 by The Exelixis Lab.
Developed by: Alexey M. Kozlov and Alexandros Stamatakis.
Contributors: Diego Darriba, Tomas Flouri, Benoit Morel, Sarah Lutteropp, Ben Bettisworth, Julia Haag, Anastasis Togkousidis.
Latest version: https://github.com/amkozlov/raxml-ng
Questions/problems/suggestions? Please visit: https://groups.google.com/forum/#!forum/raxml

System: Intel(R) Xeon(R) Silver 4110 CPU @ 2.10GHz, 16 cores, 251 GB RAM

RAxML-NG was called at 20-Aug-2025 12:11:11 as follows:

./foldtree2/raxml-ng/raxml-ng --msa famil

  0%|                                                         | 1/500 [02:26<20:20:45, 146.79s/it]

All ML trees saved to: /home/dmoi/projects/foldtree2/families/Information_benchmark/marker_genes/1428576/raxml_lg_tree.raxml.mlTrees
Optimized model saved to: /home/dmoi/projects/foldtree2/families/Information_benchmark/marker_genes/1428576/raxml_lg_tree.raxml.bestModel

Execution log saved to: /home/dmoi/projects/foldtree2/families/Information_benchmark/marker_genes/1428576/raxml_lg_tree.raxml.log

Analysis started: 20-Aug-2025 12:11:11 / finished: 20-Aug-2025 12:13:33

Elapsed time: 141.513 seconds

Inferred tree saved to ./families/Information_benchmark/marker_genes/1428576/raxml_lg_tree.raxml.bestTree
Aligning: ./families/Information_benchmark/marker_genes/966408/sequences.fasta
Creating RAxML tree for 966408 using aligned FASTA: families/Information_benchmark/marker_genes/966408/sequences.aligned.fa
Running: ./foldtree2/raxml-ng/raxml-ng --msa families/Information_benchmark/marker_genes/966408/sequences.aligned.fa --model LG+G+I --prefix ./families/Information_benchmark/marker_gen

  0%|▏                                                        | 2/500 [03:48<15:02:19, 108.71s/it]

...

[00:00:36 -6076.156351] SLOW spr round 3 (radius: 15)
[00:00:36 -6076.156349] Model parameter optimization (eps = 0.100000)

[00:00:36] ML tree search #20, logLikelihood: -6076.154030


Optimized model parameters:

   Partition 0: noname
   Rate heterogeneity: GAMMA (4 cats, mean),  alpha: 1.583218 (ML),  weights&rates: (0.250000,0.237896) (0.250000,0.601846) (0.250000,1.054957) (0.250000,2.105301) 
   P-inv (ML): 0.084032
   Base frequencies (model): 0.079066 0.055941 0.041977 0.053052 0.012937 0.040767 0.071586 0.057337 0.022355 0.062157 0.099081 0.064600 0.022951 0.042302 0.044040 0.061197 0.053287 0.012066 0.034155 0.069147 
   Substitution rates (model): 0.425093 0.276818 0.395144 2.489084 0.969894 1.038545 2.066040 0.358858 0.149830 0.395337 0.536518 1.124035 0.253701 1.177651 4.727182 2.139501 0.180717 0.218959 2.547870 0.751878 0.123954 0.534551 2.807908 0.363970 0.390192 2.426601 0.126991 0.301848 6.326067 0.484133 0.052722 0.332533 0.858151 0.578987 0.593607 0.314440 0.170887

  8%|████▊                                                     | 41/500 [55:55<8:29:22, 66.59s/it]

[00:01:38 -13668.889708] SLOW spr round 3 (radius: 15)
[00:01:38 -13668.889697] Model parameter optimization (eps = 0.100000)

[00:01:38] ML tree search #20, logLikelihood: -13668.885727


Optimized model parameters:

   Partition 0: noname
   Rate heterogeneity: GAMMA (4 cats, mean),  alpha: 1.269482 (ML),  weights&rates: (0.250000,0.187419) (0.250000,0.545075) (0.250000,1.033434) (0.250000,2.234072) 
   P-inv (ML): 0.102162
   Base frequencies (model): 0.079066 0.055941 0.041977 0.053052 0.012937 0.040767 0.071586 0.057337 0.022355 0.062157 0.099081 0.064600 0.022951 0.042302 0.044040 0.061197 0.053287 0.012066 0.034155 0.069147 
   Substitution rates (model): 0.425093 0.276818 0.395144 2.489084 0.969894 1.038545 2.066040 0.358858 0.149830 0.395337 0.536518 1.124035 0.253701 1.177651 4.727182 2.139501 0.180717 0.218959 2.547870 0.751878 0.123954 0.534551 2.807908 0.363970 0.390192 2.426601 0.126991 0.301848 6.326067 0.484133 0.052722 0.332533 0.858151 0.578987 0.593607 0.314440 0.170

  8%|████▊                                                     | 42/500 [56:24<7:01:09, 55.17s/it]

[00:00:27 -5062.596760] SLOW spr round 2 (radius: 10)
[00:00:27 -5062.596561] Model parameter optimization (eps = 0.100000)

[00:00:27] ML tree search #20, logLikelihood: -5062.589476


Optimized model parameters:

   Partition 0: noname
   Rate heterogeneity: GAMMA (4 cats, mean),  alpha: 1.469454 (ML),  weights&rates: (0.250000,0.220565) (0.250000,0.583392) (0.250000,1.048576) (0.250000,2.147467) 
   P-inv (ML): 0.084817
   Base frequencies (model): 0.079066 0.055941 0.041977 0.053052 0.012937 0.040767 0.071586 0.057337 0.022355 0.062157 0.099081 0.064600 0.022951 0.042302 0.044040 0.061197 0.053287 0.012066 0.034155 0.069147 
   Substitution rates (model): 0.425093 0.276818 0.395144 2.489084 0.969894 1.038545 2.066040 0.358858 0.149830 0.395337 0.536518 1.124035 0.253701 1.177651 4.727182 2.139501 0.180717 0.218959 2.547870 0.751878 0.123954 0.534551 2.807908 0.363970 0.390192 2.426601 0.126991 0.301848 6.326067 0.484133 0.052722 0.332533 0.858151 0.578987 0.593607 0.314440 0.170887

In [ ]:
def create_supermatrix(alignment_files, output_file , mapping_df=None):
	"""
	Create a supermatrix alignment from multiple gene alignments.
	
	Parameters:
	-----------
	alignment_files : list
		List of paths to alignment files in FASTA format
	output_file : str
		Path to save the concatenated alignment
	mapping_df : pandas.DataFrame, optional	
		DataFrame containing mapping information for structural alignments
		
	Returns:
	--------
	tuple
		(concatenated_alignment, partition_info)
		- concatenated_alignment: The final MultipleSeqAlignment object
		- partition_info: Dictionary with gene boundaries for partition file creation
	"""

	for aln in alignment_files:
		if not os.path.exists(aln):
			print(f"Alignment file {aln} does not exist, skipping")
			continue
		#read the alignment and map each aln string to an identifier
		try:
			aln_obj = AlignIO.read(aln, "fasta")
			print(f"Read alignment {aln} with {len(aln_obj)} sequences and {aln_obj.get_alignment_length()} columns")
		except Exception as e:
			print(f"Error reading alignment {aln}: {str(e)}")
			continue
		#find which line to use for each identifier from the species
		for record in aln_obj:
			spec = record.id.split("|")[1]
			if spec not in species_map:
				species_map[spec] = []
			species_map[spec].append(record)
		

In [ ]:
def run_site_likelihood_analysis(aln , tree , model,  output_prefix = None):
	"""
	Placeholder function for running site likelihood analysis.
	This function should be implemented based on specific requirements.
	"""
	print("Running site likelihood analysis...")
	#raxml command is  --force --evaluate --msa your_alignment.phy --model GTR+G --tree fixed_tree.newick --site-lh
	# Example: assumes alignment and tree files are available for each HOG
	# Example: assumes alignment and tree files are available
	if output_prefix is None:
		output_prefix = "./raxmlng_results/example"
	# Ensure output directory exists
	import os
	if os.path.exists(os.path.dirname(output_prefix)):
		print(f"Output directory {os.path.dirname(output_prefix)} already exists.")
	else:
		print(f"Creating output directory: {os.path.dirname(output_prefix)}")	
		# Create output directory if it doesn't exist
		os.makedirs(os.path.dirname(output_prefix), exist_ok=True)

	cmd = [
		"raxml-ng",
		"--force",
		"--evaluate",
		"--msa", aln,
		"--model", model,
		"--tree", tree,
		"--site-lh",
		"--prefix", output_prefix
	]
	print(f"Running: {' '.join(cmd)}")
	subprocess.run(cmd, check=True)




In [ ]:
import re
from ete3 import Tree

#aster format for trees is species_name_A for species_name and one protein. letter changes for each protein from the same species

def convert_newick_to_aster_format_ete3(newick_file, mapping_df, output_file):
	"""
	Convert a Newick tree with UniProt codes to ASTER format using mapping_df and ete3.
	Assigns a species name and a unique letter to each leaf for each protein from the same species.

	Parameters:
	-----------
	newick_file : str
		Path to input Newick file with UniProt codes as leaf names
	mapping_df : pandas.DataFrame
		DataFrame containing '1stref' (UniProt code) and 'species' (dict with 'code')
	output_file : str
		Path to output Newick file in ASTER format

	Returns:
	--------
	str: Path to the output file
	"""
	# Build mapping: UniProt -> species code
	uniprot_to_species = {}
	for _, row in mapping_df.iterrows():
		uniprot = row['1stref']
		species_code = row['species']['code'] if isinstance(row['species'], dict) else row['species']
		uniprot_to_species[uniprot] = species_code

	# Load tree using ete3
	tree = Tree(newick_file, format=1)

	# Assign letters for each protein per species
	species_letter_count = {}
	uniprot_to_aster = {}

	# Collect all leaf names
	leaf_names = [leaf.name for leaf in tree.iter_leaves()]
	for leaf in leaf_names:
		species = uniprot_to_species.get(leaf, "UNK")
		if species not in species_letter_count:
			species_letter_count[species] = 0
		letter = chr(ord('A') + species_letter_count[species])
		species_letter_count[species] += 1
		aster_name = f"{species}_{letter}"
		uniprot_to_aster[leaf] = aster_name

	# Rename leaves in the tree
	for leaf in tree.iter_leaves():
		leaf.name = uniprot_to_aster.get(leaf.name, leaf.name)

	# Write output
	tree.write(outfile=output_file, format=1)
	return output_file


In [ ]:
import subprocess
import os

#bin/astral4 -o OUTPUT_FILE INPUT_FILE 2>LOG_FILE
def run_astral(input_file, output_file, astral_path="astral.jar", log_file=None):
	"""
	Run ASTRAL to infer a species tree from gene trees.

	Parameters:
	-----------
	input_file : str
		Path to the input file containing gene trees (one per line, Newick format)
	output_file : str
		Path to save the output supertree (Newick format)
	astral_path : str
		Path to the ASTRAL jar file (default: 'astral.jar')
	log_file : str or None
		Path to save stderr log output (default: None)

	Returns:
	--------
	str: Path to the output tree file
	"""
	cmd = [
		"java", "-jar", astral_path,
		"-i", input_file,
		"-o", output_file
	]
	if log_file is not None:
		with open(log_file, "w") as logf:
			subprocess.run(cmd, stderr=logf, check=True)
	else:
		subprocess.run(cmd, check=True)
	if os.path.exists(output_file):
		print(f"ASTRAL tree saved to {output_file}")
		return output_file
	else:
		print("ASTRAL did not produce an output file.")
		return None

In [ ]:
import os
import subprocess
from pathlib import Path
import pandas as pd
import numpy as np
import re
from Bio import AlignIO

def extract_site_likelihoods(log_file):
	"""
	Extract site-wise log-likelihood values from RAxML-NG output
	
	Parameters:
	-----------
	log_file : str
		Path to the RAxML-NG log file containing site likelihoods
		
	Returns:
	--------
	list of floats: Site log-likelihood values
	"""
	likelihoods = []
	
	with open(log_file, 'r') as f:
		# Skip to the part with site likelihoods
		for line in f:
			if line.startswith('Site '):
				break
				
		# Parse the likelihood values
		for line in f:
			if not line.strip() or line.startswith('Site '):
				continue
			if 'Sum' in line:  # End of site likelihoods section
				break
				
			parts = line.strip().split()
			if len(parts) >= 2:
				try:
					likelihoods.append(float(parts[1]))
				except ValueError:
					continue
	
	return likelihoods

def create_column_likelihood_dataframe(alignment_file, tree_file, log_file,  output_dir=None):
	"""
	Calculate site likelihoods for an alignment and create a DataFrame with
	alignment columns and their corresponding likelihood values
	
	Parameters:
	-----------
	alignment_file : str
		Path to the aligned FASTA file
	tree_file : str
		Path to the tree file in Newick format
	output_dir : str or None
		Directory to store intermediate files (defaults to same directory as alignment)
		
	Returns:
	--------
	pandas.DataFrame: DataFrame with columns for site index, alignment column, and likelihood
	"""
	if output_dir is None:
		output_dir = os.path.dirname(alignment_file)
	
	# Load the alignment
	alignment = AlignIO.read(alignment_file, "fasta")
	# Extract site likelihoods
	likelihoods = extract_site_likelihoods(log_file)
	# Prepare data for DataFrame
	data = []
	for i in range(alignment.get_alignment_length()):
		if i < len(likelihoods):
			column = [record.seq[i] for record in alignment]
			column_str = ''.join(column)
			data.append({
				'Site': i + 1,
				'Alignment_Column': column_str,
				'Log_Likelihood': likelihoods[i]
			})
	# Create DataFrame
	df = pd.DataFrame(data)
	return df


In [ ]:
def alignment_character_proportions(alignment):
	"""
	Quantify the proportion of each character at every position in an alignment.

	Parameters:
	-----------
	alignment : Bio.Align.MultipleSeqAlignment
		Alignment object (e.g., from AlignIO.read)

	Returns:
	--------
	pandas.DataFrame: Rows are positions, columns are characters, values are proportions.
	"""
	columns = []
	for i in range(alignment.get_alignment_length()):
		column = [record.seq[i] for record in alignment]
		counts = Counter(column)
		total = len(column)
		proportions = {char: counts[char] / total for char in counts}
		columns.append(proportions)
	df = pd.DataFrame(columns).fillna(0)
	df.index.name = "Position"
	return df


def discretize_proportions(df, granularity=10):
	"""
	Discretize character proportions in an alignment DataFrame to specified granularity.

	Parameters:
	-----------
	df : pandas.DataFrame
		DataFrame of character proportions (output of alignment_character_proportions)
	granularity : int
		Number of bins (e.g., 10 for 0.1 steps)

	Returns:
	--------
	pandas.DataFrame: Discretized proportions (integer bins)
	"""
	bins = np.linspace(0, 1, granularity + 1)
	return df.apply(lambda col: np.digitize(col, bins, right=True), axis=0)



In [ ]:
from scipy.stats import entropy
import numpy as np

#concatenate the likelihoods and column proportions

#double integral over the likelihoods and proportions to get MI

def compute_mutual_information(proportions, likelihoods, bins=20):
	"""
	Compute mutual information between character proportions and site likelihoods.

	Parameters:
	-----------
	proportions : array-like
		Proportion values for a character across sites (e.g., column from proportions DataFrame)
	likelihoods : array-like
		Log-likelihood values for corresponding sites
	bins : int
		Number of bins to discretize both variables

	Returns:
	--------
	float: Mutual information (in nats)
	"""
	# Discretize both arrays
	prop_binned = np.digitize(proportions, np.linspace(0, 1, bins + 1), right=True)
	like_binned = np.digitize(likelihoods, np.linspace(min(likelihoods), max(likelihoods), bins + 1), right=True)

	# Joint histogram
	joint_hist, _, _ = np.histogram2d(prop_binned, like_binned, bins=bins)
	joint_prob = joint_hist / joint_hist.sum()

	# Marginals
	prop_hist = joint_hist.sum(axis=1)
	like_hist = joint_hist.sum(axis=0)
	prop_prob = prop_hist / prop_hist.sum()
	like_prob = like_hist / like_hist.sum()

	# Compute MI
	mi = 0.0
	for i in range(bins):
		for j in range(bins):
			p_ij = joint_prob[i, j]
			if p_ij > 0:
				mi += p_ij * np.log(p_ij / (prop_prob[i] * like_prob[j]))
	return mi

In [ ]:
import glob
import os
import subprocess
from pathlib import Path


for fam in 

# Directory containing encoded FASTA files
encoded_dir = './families/encoded_fastas/'  # Change to your directory
os.makedirs(encoded_dir, exist_ok=True)

# Find all encoded FASTA files
encoded_fastas = list(Path(encoded_dir).glob("*.fasta"))
print(f"Found {len(encoded_fastas)} encoded FASTA files.")

# Align each encoded FASTA file using MAFFT via treebuilder's static method
for fasta_file in encoded_fastas:
	aligned_path = str(fasta_file.with_name(f"{fasta_file.stem}.aligned.fasta"))
	tb.run_mafft_textaln(str(fasta_file), outaln=aligned_path, matrix=mafftmat)
	print(f"Aligned {fasta_file} -> {aligned_path}")




In [ ]:
#be